In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import glob
import sys
from astroquery.jplhorizons import Horizons

In [9]:
### Change as required
lc_dir = '/Users/s1523386/Documents/year1/shape_modelling/162p/LC_prep/raw/'
target = '162p'

files = sorted(glob.glob(lc_dir+target+'*R.txt'))
# Check this read correctly
if len(files)==0:
    print('Problem loading LC files, check path.')
    sys.exit()
else:
    print('%i LC files found for %s.'%(len(files), target))
    print('First file in list = ', files[0])
    print('Last file in list = ', files[-1])

18 LC files found for 162p.
First file in list =  /Users/s1523386/Documents/year1/shape_modelling/162p/LC_prep/raw/162p_20070517_950_R.txt
Last file in list =  /Users/s1523386/Documents/year1/shape_modelling/162p/LC_prep/raw/162p_20180711_809_R.txt


In [19]:
# Magnitudes found in the second column of each LC file, JD in first

mjd_start = []   # First MJD of each epoch
mjd = []   # Bit inefficient, resets at end of each file
mjd_all=[]
mag = []
mag_unc=[]

for i,file in enumerate(files):
    
    f = open(file, 'r')
    lines = f.readlines()
    for line in lines:
        data = line.split()
        mjd_all.append(float(data[0]))
        mjd.append(float(data[0]))
        mag.append(float(data[1]))
        mag_unc.append(float(data[2]))
    mjd_start.append(mjd[0])
    mjd = []
    
f.close()
len(mjd_start)

18

In [15]:
# Obtain telescope codes from files filenames to parse Horizons database
siteid = []
for file in files:
    siteid.append(file[18:21])
print(siteid)


['950', '950', '950', '309', '309', '809', '809', '309', '950', '950', '950', '071', '950', '950', '950', '809', '809', '809']


In [21]:
# Correct mid-exposure MJDs for light travel time before we obtain alphas, deltas and r_hs
# MJD_use = MJD_start-(delta/c)
au=1.496e11   #m
d2s=86400
mjd_use = []

for i,day in enumerate(mjd_start):
    obj = Horizons(id='90001062',location = siteid[i], epochs=day)
    eph = obj.ephemerides(quantities='20')
    deltcomet = float(eph['delta'])
    mjd_use.append(((day*d2s)-((deltcomet*au)/3.e8))/d2s)
mjd_use

[2454238.4504419076,
 2454239.36251031,
 2454240.3729025465,
 2456040.571266327,
 2456071.4737502034,
 2456093.490967207,
 2456096.4932989585,
 2456101.438880528,
 2457802.5503569725,
 2457803.5527208317,
 2457806.5423623323,
 2457811.504141046,
 2458219.455531825,
 2458220.4885629457,
 2458227.435531547,
 2458309.3739130935,
 2458310.374203054,
 2458311.370768656]

In [29]:
# Use light-time corrected MJDs to parse Horizons for delta, alpha, r_h
r_au = []
delta_au = []
alpha = []

for i in range(len(siteid)):
    obj = Horizons(id='90001064',location = siteid[i], epochs=mjd_use[i])
    eph = obj.ephemerides(quantities='19,20')
    
    r = float(eph['r'])
    r_au.append(r)
    
    delta = float(eph['delta'])
    delta_au.append(delta)


In [25]:
# Defining funciton to calculate reduced magnitude H(1,1,alpha)
# Inputs:
#    m: apparent magnitude
#    r: heliocentric distance (AU)
#    delta: geocentric distance (AU)

# Outputs:
#    h: H(1,1,alpha)
def mag_reduce(m, r, delta):
    h = m-(5*np.log10(r*delta))
    return h

In [42]:
count = 0
for i,day in enumerate(mjd_all):
    if (day in mjd_start):
        count = count+1
    print(count, day, mag_reduce(mag[i], r_au[count-1], delta_au[count-1]), mag_unc[i])

1 2454238.473696 13.965962817245378 0.0165
1 2454238.475177 13.96996281724538 0.017
1 2454238.47667 13.948962817245379 0.0161
1 2454238.478152 13.95896281724538 0.0174
1 2454238.479633 13.951962817245379 0.0188
1 2454238.481115 13.94096281724538 0.0209
1 2454238.482596 13.985962817245378 0.0195
1 2454238.484078 13.938962817245377 0.0203
1 2454238.485582 13.999962817245377 0.0185
1 2454238.487064 13.96396281724538 0.0197
1 2454238.549216 13.973962817245377 0.0221
1 2454238.553696 13.980962817245379 0.0257
1 2454238.558152 13.98196281724538 0.0336
2 2454239.385819 14.23461124499773 0.0364
2 2454239.387543 14.203611244997727 0.0352
2 2454239.389256 14.239611244997729 0.0292
2 2454239.390993 14.20561124499773 0.0262
2 2454239.392705 14.19511124499773 0.0251
2 2454239.399418 14.23161124499773 0.0271
2 2454239.404476 14.299611244997728 0.0221
2 2454239.551745 14.41161124499773 0.0353
2 2454239.553469 14.395111244997729 0.0377
2 2454239.601224 14.41111124499773 0.0706
2 2454239.602717 14.3276

13 2458219.652654 13.774106916303671 0.0383
13 2458219.654036 13.87780691630367 0.0384
13 2458219.655415 13.85070691630367 0.0313
13 2458219.656786 13.807106916303669 0.0381
13 2458219.658152 13.821006916303668 0.0372
13 2458219.662955 13.75010691630367 0.0471
13 2458219.664339 13.79300691630367 0.0353
13 2458219.672585 13.780106916303671 0.0314
13 2458219.673962 13.747706916303668 0.0325
13 2458219.675328 13.81860691630367 0.0332
13 2458219.676705 13.785906916303668 0.0314
13 2458219.678083 13.814606916303669 0.0352
13 2458219.679472 13.78640691630367 0.0363
13 2458219.680837 13.762806916303669 0.0334
13 2458219.682214 13.775506916303671 0.0344
13 2458219.683592 13.75290691630367 0.0324
13 2458219.684957 13.74130691630367 0.0341
13 2458219.68718 13.76230691630367 0.0324
13 2458219.68857 13.73170691630367 0.0332
13 2458219.689935 13.74070691630367 0.0316
14 2458220.51107 13.68951726310293 0.0402
14 2458220.512562 13.75931726310293 0.0454
14 2458220.514332 13.746817263102928 0.0372
14 2